# Multitenancy

When building a retrieval app, you often have to build it with mulit-tenancy in mind. This means that you may be storing data not just for one user, but for many different users, and they should not be able to see eachother's data. This means that you need to be able to configure your retrieval chain to only retrieve certain information. This generally involves two steps.

**Step 1: Make sure the retriever you are using supports multi-tenancy**

At the moment, there is no unified flag or filter fo multi-tenancy in LangChain. Rather, each vectorstore and retriever may have their own. For vectorstores, this is generally exposed as a keyword argument that is passed in during `similarity_search`. By reading the documentation or source code, figure out whether the retriever you are using supports multi-tenancy, and, if so, how to use it.

Note: adding documentation and/or support for multi-tenancy for retrievers that do not support it (or document it) is a GREAT way to contribute to LangChain

**Step 2: Add that parameter as a configurable field for the chain**

This will let you easily call the chain and configure any multi-tenancy flags at runtime. See [this documentation](docs/expression_language/how_to/configure) for more information on configuration.

**Step 3: Call the chain with that configurable field**

Now, at runtime you can call this chain with configurable field.

## Code Example

Let's see a concrete example of what this looks like in code. We will use Pinecone for this example.

In [1]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/Users/harrisonchase/.pyenv/versions/3.10.1/envs/langchain/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
# The environment should be the one specified next to the API key
# in your Pinecone console
pinecone.init(
    api_key="...", environment="..."
)
index = pinecone.Index("test-example")
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone(index, embeddings, "text")

vectorstore.add_texts(["i worked at kensho"], namespace="harrison")
vectorstore.add_texts(["i worked at facebook"], namespace="ankush")

['ce15571e-4e2f-44c9-98df-7e83f6f63095']

The pinecone kwarg for `namespace` can be used to separate documents

In [6]:
# This will only get documents for Ankush
vectorstore.as_retriever(search_kwargs={"namespace": "ankush"}).get_relevant_documents(
    "where did i work?"
)

[Document(page_content='i worked at facebook')]

In [7]:
# This will only get documents for Harrison
vectorstore.as_retriever(search_kwargs={"namespace": "harrison"}).get_relevant_documents(
    "where did i work?"
)

[Document(page_content='i worked at kensho')]

We can now create the chain that we will use to do question-answering over

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableBinding
from langchain_core.runnables import ConfigurableField

This is basic question-answering chain set up.

In [10]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retriever = vectorstore.as_retriever()

Here we mark the retriever as having a configurable field. All vectorstore retrievers have `search_kwargs` as a field. This is just a dictionary, with vectorstore specific fields

In [11]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

We can now create the chain using our configurable retriever

In [12]:
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

We can now invoke the chain with configurable options. `search_kwargs` is the id of the configurable field. The value is the search kwargs to use for Pinecone

In [13]:
chain.invoke(
  "where did the user work?", 
  config={"configurable": {"search_kwargs": {"namespace": "harrison"}}}
)

'The user worked at Kensho.'

In [14]:
chain.invoke(
  "where did the user work?", 
  config={"configurable": {"search_kwargs": {"namespace": "ankush"}}}
)

'The user worked at Facebook.'